# Private Support Points: Regression Examples

___

In [ ]:
import json
import logging
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
import os
import pdb
import sklearn
from sklearn.datasets import load_boston, load_diabetes, fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split
import time

from sp_utils import (
    scale_01,
    get_energy_sensitivity,
    get_support_points,
    energy,
    sample_sp_exp_mech,
    sample_sp_mmd_dp_bw,
    mixture_model_likelihood,
    sample_full_set_given_bandwidth)

plt.style.use('ggplot')
font = {'family' : 'normal',
        'size'   : 14}
matplotlib.rc('font', **font)

In [ ]:
# Set up logging.
log_filename = '../output/regression_logs/run.log'
if os.path.isfile(log_filename):
    os.remove(log_filename)
logging.basicConfig(filename=log_filename,
                             filemode='a',
                             format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                             datefmt='%H:%M:%S',
                             level=logging.DEBUG)
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING)
_LOG = logging.getLogger('[perturbed]')

# Support points accurately model toy multivariate data in 5D.

In [ ]:
def plot_nd(d, w=10, h=10, title=None):
    graph = pd.plotting.scatter_matrix(pd.DataFrame(d), figsize=(w, h));
    if title:
        plt.suptitle(title)
    plt.show()

In [ ]:
# Test support points on multivariate Gaussian data.
if 0:
    M = 500
    D = 5
    data = np.random.normal(0, 1, size=(M, D))
    N = 100
    MAX_ITER = 501 #300
    LR = 5e-1 #1e1

    y_opt, e_opt = get_support_points(data, N, MAX_ITER, LR, is_tf=True,
                                      Y_INIT_OPTION='uniform', clip='data')

    plot_nd(pd.DataFrame(data), 5, 5, 'data')
    plot_nd(pd.DataFrame(y_opt), 5, 5, 'sp')

# Test variance of SP vs variance of Diffusion Sampling.

In [ ]:
M = 500
N = 50
dim = 2
MAX_ITER = 201
LR = 5e-1
ENERGY_POWER = 2
ALPHA = 10

num_sp_samples = 3

# Make data.
mean = [0, 0]
cov = [[1, 0.5], [0.5, 1]]
data = np.random.multivariate_normal(mean, cov, [M])
data = scale_01(data)

if 0:

    # ------------------------------------------------------------------
    # Given data, repeated support point optimizations are low variance.
    # ------------------------------------------------------------------

    plt.scatter(data[:, 0], data[:, 1], c='gray', s=64,
                alpha=0.3, label='data')
    for i in range(num_sp_samples):
        y_opt, e_opt = get_support_points(data, N, MAX_ITER, LR, is_tf=True,
                                          Y_INIT_OPTION='uniform', clip='data',
                                          plot=False)
        plt.scatter(y_opt[:, 0], y_opt[:, 1], s=32, label='sp_{}'.format(i))

    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.show()


    # ---------------------------------------------------------------
    # Given support points, repeated diffusions create more variance.
    # ---------------------------------------------------------------

    # Get private support points.
    y_opt, e_opt = get_support_points(data, N, MAX_ITER, LR, is_tf=True,
                                      Y_INIT_OPTION='uniform', clip='data',
                                      plot=False)
    energy_sensitivity = 2 * dim ** (1. / ENERGY_POWER) * (2 * N - 1) / N ** 2
    (y_tildes_diffusion,
     energies_diffusion,
     energy_errors_diffusion) = sample_sp_exp_mech(e_opt, energy_sensitivity,
                                                   data, y_opt, 'mh',
                                                   step_size=0.01,
                                                   num_y_tildes=num_sp_samples,
                                                   alpha=ALPHA,
                                                   diffusion_mean=True)

    plt.scatter(data[:, 0], data[:, 1], c='gray', s=64,
                alpha=0.3, label='data')
    plt.scatter(y_opt[:, 0], y_opt[:, 1], s=32, c='limegreen',
                label='sp')
    for i in range(num_sp_samples):
        y_priv = y_tildes_diffusion[i]
        plt.scatter(y_priv[:, 0], y_priv[:, 1], s=32, label='~sp_{}'.format(i))
    plt.title(('|data|={}, |supp|={}, eps={:.3f}, e_opt: {:.8f}').format(
                   M, N, ALPHA, e_opt))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend()
    plt.show()

___

# Regression task.

In [ ]:
def test_regression(data, data_heldout):
    """Computes regression accuracy, averaged over multiple runs.
    
    Each run computes accuracy with k-fold cross validation.
    
    Args:
      data (array): Training data.
      data_heldout (array): Testing/heldout data.
    
    Returns:
      result (scalar): MSE value on test data.
    """
    
    def regress(X_train, X_test, Y_train, Y_test):
        # Fits linear model given a train-test split.
        # Returns MSE value of fitted model.
        lm = LinearRegression()
        lm.fit(X_train, Y_train)

        Y_pred = lm.predict(X_test)
        mse = mean_squared_error(Y_test, Y_pred)
        r2 = lm.score(X_test, Y_test)
        
        return mse
    
        
    result = regress(data[:, :-1], data_heldout[:, :-1],
                     data[:, -1], data_heldout[:, -1])
    
    return result

## Helper functions for running experiments.

In [ ]:
from collections import namedtuple
Result = namedtuple('Result', 'dataset_name size alpha mse std tag')


def visualize_data(data, title=None):
    # Visualize data with pairs plot.
    if len(data) >= 1000:
        _d = data[np.random.choice(len(data), 500)]             
        plot_nd(_d, 10, 10, title)
    else:
        plot_nd(data, 10, 10, title)


def round_binary_cols(data, binary_cols):
    """Rounds binary cols of a NumPy array."""
    for col in binary_cols:
        col_data = data[:, col]
        col_data = np.clip(col_data, 0, 1)
        col_data = np.round(col_data)
        data[:, col] = col_data
    return data


def get_accuracy_mean_std_on_heldout(candidate_sets, heldout, data,
                                     dataset_name, N, alpha=None, tag=None):
    """Computes regression accuracy results for a multiset of candidate points.
    
    Multiset can be random subsets of data, can be sets of support points, or
    can be sets of privatized support points.

    Args:
        candidate_sets (array): Multiset of candidate points.
        heldout (array): Array of test data.
        data (array): Data for plotting.
        dataset_name (string): Name of data set.
        N (int): Number of support points.
        alpha (float): Privacy budget.
        tag (string): Tag for Result object.

    Returns:
        result_priv (Result): Named tuple of result for private support points.
    """
    # Collect accuracy values for each sample.
    accuracies = []
    for candidate_set in candidate_sets:
        accuracy = test_regression(candidate_set, heldout)
        accuracies.append(accuracy)
    _LOG.info('accuracies_min_max = {}, {}'.format(min(accuracies),
                                                   max(accuracies)))
    _LOG.info('accuracies_std = {}'.format(np.std(accuracies)))

    # Mean, Std of accuracies across samples.
    accuracy_mean = np.round(np.mean(accuracies), 4)
    accuracy_std = np.round(np.std(accuracies), 4)

    # Store results of sample runs for this (N, alpha) combination.
    result = Result(dataset_name, N, alpha, accuracy_mean, accuracy_std, tag)

    # Visualize data versus sample of private support points.
    #visualize_data(data, 'data')
    #visualize_data(candidate_sets[-1], 'N={}, alpha={}'.format(N, alpha))

    return result
                

def resample_from_histdd(H, edge_sets, n=100, plot=False):
    """Resamples data set from histogram, uniformly over bins.
    
    Args:
        H (array): Arrays of counts per bin.
        edge_sets (array): Arrays of edge boundaries per dim.
        n (int): Number of points to sample.

    Returns:
        resampled (array): Newly sampled data.
    """
    bin_widths = [np.diff(edges)[0] for edges in edge_sets]
    midpoints = [edges[:-1] + np.diff(edges) / 2 for edges in edge_sets]

    # Compute CDF of counts, then normalize.
    cdf = np.cumsum(H.ravel())
    cdf = cdf / cdf[-1]
    
    # Sample uniform, associate to CDF values.
    values = np.random.rand(n)
    value_bins = np.searchsorted(cdf, values)
    
    # Fetch associated indices from original grid.
    unraveled_shape = [len(r) for r in midpoints]
    hist_indices = np.array(np.unravel_index(value_bins, unraveled_shape))
    
    # Sample uniformly on bin.
    random_from_cdf = []
    num_dims = len(hist_indices)
    for i in range(num_dims):
        bin_width_i = bin_widths[i]
        mids_i = midpoints[i][hist_indices[i]]
        vals_i = mids_i + np.random.uniform(low=-bin_width_i / 2,
                                            high=bin_width_i / 2,
                                            size=mids_i.shape)
        random_from_cdf.append(vals_i)
    resampled = np.array(random_from_cdf).T
    #random_from_cdf = np.array([midpoints[i][hist_indices[i]] for i in range(num_dims)])
    
    # Visualize data.
    if plot:
        visualize_data(resampled, 'resampled')
        
    return resampled
    
    
def run_experiments(dataset_name, data_orig, num_supp, alphas,
                    max_iter, lr, num_sp_samples, step_size, method='mh'):
    """Runs panel of experiments for different number of support points
    and different alpha settings.
    
    Args:
      dataset_name: String name.
      data_orig: NumPy array of data. NOTE: Target var must be last column [!].
      num_supp: List of support point set sizes.
      alphas: List of alphas.
      max_iter: Int, number of steps in support point optimization.
      lr: Float, learning rate of support point optimization.
      num_sp_samples: Int, number of Support Point sets over which 
        to average regression performance.
      step_size: Float, step size of support point sampling.
      method: String, name of private-sampling method. ['diffusion', 'mh']
      
    Returns:
      results: List of Result objects.
    """

    results = []

    # Fetch, scale, and shuffle data.
    data_scaled = data_orig
    assert np.min(data_scaled) >= 0 and np.max(data_scaled) <= 1, 'Scaling incorrect.'
    np.random.shuffle(data_scaled)

    # Create folds for cross validation.
    kf = KFold(n_splits=2)

    # Do an experiment run for each train-test split.
    for train_index, test_index in kf.split(data_scaled):
        _LOG.info('Starting new data split.')
        
        if max(max(train_index), max(test_index)) >= len(data_scaled):
            pdb.set_trace()
        data = data_scaled[train_index]
        data_heldout = data_scaled[test_index]
    
        visualize_data(data)
    
        # --------------------------------------
        # Test regression on FULL TRAINING data.
        # --------------------------------------
        result_training_data = get_accuracy_mean_std_on_heldout(
            [data], data_heldout,
            data, dataset_name, len(data), 
            alpha=None, tag='full_training')
        results.append(result_training_data)
        _LOG.info(result_training_data)


        # --------------------------------------------
        # Test regression on PERTURBED HISTOGRAM data.
        # --------------------------------------------

        for alpha in alphas:
            n, d = data.shape
            num_bins = (n * alpha / 10) ** ((2 * d) / (2 + d))
            num_bins_per_dim = int(np.round(num_bins ** (1 / d)))  # https://arxiv.org/pdf/1504.05998.pdf

            # Get true histogram, and perturb with Laplace noise.
            H, edges = np.histogramdd(data, bins=num_bins_per_dim)

            # Perturb histogram counts with Laplace noise.
            H_perturbed = H + np.random.laplace(loc=0, scale=1/alpha, size=H.shape)

            # Resample from perturbed histogram, using uniform sampling per bin.
            perturbed_resample = resample_from_histdd(H_perturbed, edges, n=n)

            # Evaluate regression performance on heldout data.
            result_perturbed_resample = get_accuracy_mean_std_on_heldout(
                [perturbed_resample], data_heldout,
                perturbed_resample, dataset_name, len(perturbed_resample), 
                alpha=alpha, tag='perturbed_resample')
            results.append(result_perturbed_resample)
            _LOG.info(result_perturbed_resample)


        # -------------------------------------------

        for N in num_supp:

            # -------------------------------------------
            # Test regression on RANDOM SUBSETS (size N).
            # -------------------------------------------
            random_subsets_data = [
                data[np.random.choice(len(data), N)]
                for i in range(num_sp_samples)]
            result_random_subsets_data = get_accuracy_mean_std_on_heldout(
                random_subsets_data, data_heldout,
                data, dataset_name, N, alpha=None, tag='random_subset')
            results.append(result_random_subsets_data)
            _LOG.info(result_random_subsets_data)


            # ------------------------------------------------------
            # Test regression on REPEATED SP optimizations (size N).
            # ------------------------------------------------------
            sp_sets = []
            for i in range(num_sp_samples):
                y_opt, e_opt = get_support_points(data, N, max_iter, lr, is_tf=True,
                                                  Y_INIT_OPTION='uniform', clip='data',
                                                  plot=False)
                sp_sets.append(y_opt)
            result_sp_sets = get_accuracy_mean_std_on_heldout(
                sp_sets, data_heldout, data,
                dataset_name, N, alpha=None, tag='support_points')
            results.append(result_sp_sets)
            _LOG.info(result_sp_sets)


            # ---------------------------------------------------
            # Test regression on PRIVATE SUPPORT POINTS (size N).
            # ---------------------------------------------------

            # For each alpha, compute private support points and test regression on them.
            for alpha in alphas:
                print(('\n\n---------------------------'
                       '\nRunning for N={}, alpha={}').format(N, alpha))
                _LOG.info('Running for N={}, alpha={}'.format(N, alpha))

                # TODO: TESTING FIXED STEP SIZES BASED ON ALPHA.
                if alpha <= 1:
                    step_size = 0.05
                elif alpha <= 10:
                    step_size = 0.01
                else:
                    step_size = 0.005


                # -------------------------------   
                # Compute private support points.
                # -------------------------------   

                energy_sensitivity = get_energy_sensitivity(data, N, alpha)
                print(('Exp(2 * U / alpha) = Exp(2 * {:.4f} / {:.2f}) '
                       '= Exp({:.3f})').format(energy_sensitivity, alpha, 
                                               2. * energy_sensitivity / alpha))
                _LOG.info(('Exp(2 * U / alpha) = Exp(2 * {:.4f} / {:.2f}) '
                           '= Exp({:.3f})').format(energy_sensitivity, alpha, 
                                                   2. * energy_sensitivity / alpha))

                # For this (N, alpha) combo, run sampler many times.
                # Resulting accuracy values will be averaged.
                (y_tildes,
                 energies,
                 energy_errors) = sample_sp_exp_mech(e_opt, energy_sensitivity,
                                                     data, y_opt, method,
                                                     step_size=step_size,
                                                     num_y_tildes=num_sp_samples,
                                                     alpha=alpha,
                                                     diffusion_mean=True)

                private_sps = y_tildes


                # ------------------------------------------   
                # Test regression on private support points.
                # ------------------------------------------

                result = get_accuracy_mean_std_on_heldout(private_sps,
                                                          data_heldout,
                                                          data,
                                                          dataset_name, N, alpha=alpha,
                                                          tag='private_support_points')
                results.append(result)
                _LOG.info(result)



            
    return results  

In [ ]:
def dict_filter_tag(rows, query, dataset, tag):
    """Gets results. Note: As saved, ordered by increasing N."""
    r = [d[query] for d in rows if
         d['dataset_name'] == dataset and
         d['tag'] == tag]
    return r

def dict_filter_alpha(rows, query, dataset, alpha):
    """Gets results. Note: As saved, ordered by increasing N."""
    r = [d[query] for d in rows if
         d['dataset_name'] == dataset and
         d['tag'] == 'private_support_points' and
         d['alpha'] == alpha]
    return r

In [ ]:
def plot_final_results(results, dataset_name, alphas):
    """Plots regression results.
    
    Args:
      results (list): List of Result objects.
      dataset_name (string): Name of dataset, e.g. 'boston'.
      alphas (list): List of alphas used in experiment panel.
    
    Returns:
      None
    """
    
    sizes = np.unique([d.size for d in results if d.tag == 'random_subset'])
    x = np.linspace(0.2, 1, 5)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15,8))
    fig.suptitle(dataset_name)
    
    # -------------------------------------
    # Make boxplot for fixed size elements.
    # -------------------------------------
    
    _full_training = [d.mse for d in results if d.tag == 'full_training']
    _alpha_results = []
    for a in alphas:
        _pert_a = [
            d.mse for d in results if
            d.tag == 'perturbed_resample' and
            d.alpha == a]
        _alpha_results.append(_pert_a)
        
    ax1.boxplot(np.column_stack([_full_training] + _alpha_results),
                labels=['train'] + ['pert:{}'.format(a) for a in alphas])

    
    """    
    _pert100 = [
        d['mse'] for d in results if
        d['tag'] == 'perturbed_resample' and
        d['alpha'] == 100]
    _pert10 = [
        d['mse'] for d in results if
        d['tag'] == 'perturbed_resample' and
        d['alpha'] == 10]
    _pert1 = [
        d['mse'] for d in results if
        d['tag'] == 'perturbed_resample' and
        d['alpha'] == 1]
    ax1.boxplot(np.column_stack((_full_training, _pert100, _pert10, _pert1)),
                labels=['train', r'pert:100', r'pert:10', r'pert:1'])
    """
    
    
    # ------------------------------------
    # Then make line plot with error bars.
    # ------------------------------------
    
    # Random subset.
    mses = []
    stds = []
    for size in sizes:
        res = [
            d.mse for d in results if
            d.size == size and
            d.tag == 'random_subset']
        mses.append(np.mean(res))
        stds.append(np.std(res))
   
    ax2.errorbar(x, mses, yerr=stds, label='random_subset')

    # Support points.
    mses = []
    stds = []
    for size in sizes:
        res = [
            d.mse for d in results if
            d.size == size and
            d.tag == 'support_points']
        mses.append(np.mean(res))
        stds.append(np.std(res))
    
    ax2.errorbar(x + 0.01, mses, yerr=stds, label='support_points')
    
    # Private support points.
    for i, alpha in enumerate(alphas):
        mses = []
        stds = []
        for size in sizes:
            res = [d.mse for d in results if
                   d.size == size and
                   d.tag == 'private_support_points' and
                   d.alpha == alpha]
            mses.append(np.mean(res))
            stds.append(np.std(res))

        ax2.errorbar(x + 0.02 + i * 0.01,
                     mses,
                     yerr=stds,
                     label=r'sp, $\alpha={}$'.format(alpha))

    # ------------------------------------    
    
    ax1.set_xlabel('Data used for fitting')    
    ax2.set_xlabel('Number of Points, Fraction of Whole')
    ax1.set_ylabel('Mean Squared Error')

    ax2.legend()
    plt.savefig('../output/regression_global_results_{}'.format(dataset_name))
    plt.show()

___

# Full Experiments.

In [ ]:
run_boston = 1
run_diabetes = 1
run_california = 1

alphas = [1000, 100, 10, 1]
num_sp_samples = 10
num_cv_splits = 5

global_results = []

## BOSTON

In [ ]:
# Get Boston data.
dataset_name = 'boston'
dataset = load_boston()
d = pd.DataFrame(dataset.data)
d.columns = dataset.feature_names
d['MEDV'] = dataset.target

data = np.concatenate((dataset.data, dataset.target.reshape(-1, 1)), axis=1)
data = scale_01(data)
np.random.shuffle(data)

# Optional subsetting for troubleshooting.
#data = data[np.random.choice(len(data), 200, replace=False)]
#data = data[:, [1, 2, 3]]
#plot_nd(data, title='Data, scaled [0, 1]')

In [ ]:
# Define set of parameters to test.
num_supp = [int(len(data) / num_cv_splits * i / 10) for i in range(2, 11, 2)] 
max_iter = 301
lr = 0.5
step_size = 1e-3

In [ ]:
# Run experiments and print results.
if run_boston:
    np.random.seed(123)
    results = run_experiments(dataset_name, data, num_supp, alphas,
                              max_iter, lr, num_sp_samples, step_size)
    
    global_results.extend(results)

In [ ]:
if run_boston:
    plot_final_results(results, 'boston', alphas)

## DIABETES

In [ ]:
# Get diabetes data.
dataset_name = 'diabetes'
dataset = load_diabetes()
d = pd.DataFrame(dataset.data)
d.columns = dataset.feature_names
d['TARGET'] = dataset.target

data = np.concatenate((dataset.data, dataset.target.reshape(-1, 1)), axis=1)
data = scale_01(data)
np.random.shuffle(data)

# Optional subsetting for troubleshooting.
#data = data[np.random.choice(len(data), 200, replace=False)]
#data = data[:, [1, 2, 3]]
#plot_nd(data, title='Data, scaled [0, 1]')

In [ ]:
# Define set of parameters to test.
num_supp = [int(len(data) / num_cv_splits * i / 10) for i in range(2, 11, 2)] 
max_iter = 301
lr = 0.5
step_size = 1e-3

In [ ]:
# Run experiments and print results.
if run_diabetes:
    np.random.seed(123)
    results = run_experiments(dataset_name, data, num_supp, alphas,
                              max_iter, lr, num_sp_samples, step_size)
    
    global_results.extend(results)

In [ ]:
if run_diabetes:
    plot_final_results(results, 'diabetes', alphas)

## CALIFORNIA

In [ ]:
# Get diabetes data.
dataset_name = 'california'
dataset = fetch_california_housing()
d = pd.DataFrame(dataset.data)
d.columns = dataset.feature_names
d['TARGET'] = dataset.target

data = np.concatenate((dataset.data, dataset.target.reshape(-1, 1)), axis=1)
data = scale_01(data)
np.random.shuffle(data)

# Optional subsetting for troubleshooting.
#data = data[np.random.choice(len(data), 200, replace=False)]
#data = data[:, [1, 2, 3]]
#plot_nd(data, title='Data, scaled [0, 1]')

In [ ]:
# Define set of parameters to test.
num_supp = [int(len(data) / num_cv_splits * i / 10) for i in range(2, 11, 2)] 
max_iter = 501
lr = 1.
step_size = 5e-4

In [ ]:
# Run experiments and print results.
if run_california:
    np.random.seed(123)
    results = run_experiments(dataset_name, data, num_supp, alphas,
                              max_iter, lr, num_sp_samples, step_size)

    global_results.extend(results)
    plot_final_results(results, 'california', alphas)

In [ ]:
if run_california:
    plot_final_results(results, 'california', alphas)

___

# Save and plot global results

In [ ]:
# Write results to file.
import json
gr_as_dict = [row._asdict() for row in global_results]
with open('../output/regression_global_results.json', 'w') as outfile:
    json.dump(gr_as_dict, outfile)

___